# Challenge 2

In this challenge we will continue working with the `Pokemon` dataset. We will attempt solving a slightly more complex problem in which we will practice the iterative data analysis process you leaned in [this video](https://www.youtube.com/watch?v=xOomNicqbkk).

The problem statement is as follows:

**You are at a Pokemon black market planning to buy a Pokemon for battle. All Pokemon are sold at the same price and you can only afford to buy one. You cannot choose which specific Pokemon to buy. However, you can specify the type of the Pokemon - one type that exists in either `Type 1` or `Type 2`. Which type should you choose in order to maximize your chance of receiving a good Pokemon?**

To remind you about the 3 steps of iterative data analysis, they are:

1. Setting Expectations
1. Collecting Information
1. Reacting to Data / Revising Expectations

Following the iterative process, we'll guide you in completing the challenge.


## Problem Solving Iteration 1

In this iteration we'll analyze the problem and identify the breakthrough. The original question statement is kind of vague because we don't know what a *good pokemon* really means as represented in the data. We'll start by understanding the dataset and see if we can find some insights.

In [2]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

From the data it seems whether a pokemon is good depends on its abilities as represented in the fields of `HP`, `Attack`, `Defense`, `Sp. Atk`, `Sp. Def`, `Speed`, and `Total`. We are not sure about `Generation` and `Legendary` because they are not necessarily the decisive factors of the pokemon abilities.

But `HP`, `Attack`, `Defense`, `Sp. Atk`, `Sp. Def`, `Speed`, and `Total` are a lot of fields! If we look at them all at once it's very complicated. This isn't Mission Impossible but it's ideal that we tackle this kind of problem after we learn Machine Learning (which you will do in Module 3). For now, is there a way to consolidate the fields we need to look into?

Fortunately there seems to be a way. It appears the `Total` field is computed based on the other 6 fields. But we need to prove our theory. If we can approve there is a formula to compute `Total` based on the other 6 abilities, we only need to look into `Total`.

We have the following expectation now:

#### The `Total` field is computed based on `HP`, `Attack`, `Defense`, `Sp. Atk`, `Sp. Def`, and `Speed`.

We need to collect the following information:

* **What is the formula to compute `Total`?**
* **Does the formula work for all pokemon?**

In the cell below, make a hypothesis on how `Total` is computed and test your hypothesis.

In [46]:
# your code here
pokemon = pd.read_csv(r'C:\Users\river\Ironhack-data\semana2\19_04_23\2.3-lab_df_calculation_and_transformation\your-code\Pokemon.csv')

pokemon

(pokemon['Total'] == pokemon['HP'] + pokemon['Attack'] + pokemon['Defense'] + pokemon['Sp. Atk'] + pokemon['Sp. Def'] + pokemon['Speed']).any()

# al ejecutar este codigo encuentr que algún valor no coincide por lo tanto esta formula no sirve para todos los pokemon

no_cumple = pokemon[pokemon['Total'] != (pokemon['HP'] + pokemon['Attack'] + pokemon['Defense'] + pokemon['Sp. Atk'] + pokemon['Sp. Def'] + pokemon['Speed'])]

no_cumple

# sale vacia así que la formula funciona para todos los pokemons




,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary


## Problem Solving Iteration 2

Now that we have consolidated the abilities fields, we can update the problem statement. The new problem statement is:

### Which pokemon type is most likely to have the highest `Total` value?

In the updated problem statement, we assume there is a certain relationship between the `Total` and the pokemon type. But we have two *type* fields (`Type 1` and `Type 2`) that have string values. In data analysis, string fields have to be transformed to numerical format in order to be analyzed. 

In addition, keep in mind that `Type 1` always has a value but `Type 2` is sometimes empty (having the `NaN` value). Also, the pokemon type we choose may be either in `Type 1` or `Type 2`.

Now our expectation is:

#### `Type 1` and `Type 2` string variables need to be converted to numerical variables in order to identify the relationship between `Total` and the pokemon type.

The information we need to collect is:

#### How to convert two string variables to numerical?

Let's address the first question first. You can use a method called **One Hot Encoding** which is frequently used in machine learning to encode categorical string variables to numerical. The idea is to gather all the possible string values in a categorical field and create a numerical field for each unique string value. Each of those numerical fields uses `1` and `0` to indicate whether the data record has the corresponding categorical value. A detailed explanation of One Hot Encoding can be found in [this article](https://hackernoon.com/what-is-one-hot-encoding-why-and-when-do-you-have-to-use-it-e3c6186d008f). You will formally learn it in Module 3.

For instance, if a pokemon has `Type 1` as `Poison` and `Type 2` as `Fire`, then its `Poison` and `Fire` fields are `1` whereas all other fields are `0`. If a pokemon has `Type 1` as `Water` and `Type 2` as `NaN`, then its `Water` field is `1` whereas all other fields are `0`.

#### In the next cell, use One Hot Encoding to encode `Type 1` and `Type 2`. Use the pokemon type values as the names of the numerical fields you create.

The new numerical variables you create should look like below:

![One Hot Encoding](../images/one-hot-encoding.png)

In [71]:
# your code here
pokemon_dummies=pd.get_dummies(pokemon['Type 1'],dtype=int)
pokemon_dummies_2=pd.get_dummies(pokemon['Type 2'],dtype=int)
pokemon_dummies_total=pokemon_dummies+pokemon_dummies_2
pokemon_dummies_total


,Bug,Dark,Dragon,Electric,Fairy,Fighting,Fire,Flying,Ghost,Grass,Ground,Ice,Normal,Poison,Psychic,Rock,Steel,Water
0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0
1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0
2,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0
3,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0
4,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0
796,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0
797,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0
798,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0


## Problem Solving Iteration 3

Now we have encoded the pokemon types, we will identify the relationship between `Total` and the encoded fields. Our expectation is:

#### There are relationships between `Total` and the encoded pokemon type variables and we need to identify the correlations.

The information we need to collect is:

#### How to identify the relationship between `Total` and the encoded pokemon type fields?

There are multiple ways to answer this question. The easiest way is to use correlation. In the cell below, calculate the correlation of `Total` to each of the encoded fields. Rank the correlations and identify the #1 pokemon type that is most likely to have the highest `Total`.

In [119]:
# your code here
pokemon_dummies_total['Total']=pokemon['Total']
matriz_pokemon_corr=pokemon_dummies_total.corr()
df_matriz_pokemon_corr=pd.DataFrame(matriz_pokemon_corr['Total'])


df_matriz_pokemon_corr



#los que más interesan son dragon y psychic

,Total
Bug,-0.145781
Dark,0.056154
Dragon,0.229705
Electric,0.020971
Fairy,-0.036698
Fighting,0.077786
Fire,0.078726
Flying,0.059383
Ghost,0.003641
Grass,-0.052592


# Bonus Question

Say now you can choose both `Type 1` and `Type 2` of the pokemon. In order to receive the best pokemon, which types will you choose?

In [121]:
# your code here

df_matriz_pokemon_corr.sort_values(by=['Total'], ascending = False)

display(df_matriz_pokemon_corr.sort_values(by=['Total'], ascending = False))

#añadir si existe la casuistica de tipo 1 dragon y tipo 2 psychic

pokemon[(pokemon['Type 1'] == 'Dragon') & (pokemon['Type 2'] == 'Psychic')]

#por lo tanto si que existe este pokemon que cumple con la condicion de tener el tipo 1 dragon y tipo 2 psychic


,Total
Total,1.000000
Dragon,0.229705
Psychic,0.124688
Steel,0.109703
Fire,0.078726
Fighting,0.077786
Ice,0.060248
Flying,0.059383
Dark,0.056154
Rock,0.032731


,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary,TYPE 1
85,79,Slowpoke,Dragon,Psychic,315,90,65,65,40,40,15,1,False,Dragon
86,80,Slowbro,Dragon,Psychic,490,95,75,110,100,80,30,1,False,Dragon
87,80,SlowbroMega Slowbro,Dragon,Psychic,590,95,75,180,130,80,30,1,False,Dragon
110,102,Exeggcute,Dragon,Psychic,325,60,40,80,60,45,40,1,False,Dragon
111,103,Exeggutor,Dragon,Psychic,520,95,95,85,125,65,55,1,False,Dragon
130,121,Starmie,Dragon,Psychic,520,60,75,85,100,85,115,1,False,Dragon
133,124,Jynx,Dragon,Psychic,455,65,50,35,115,95,95,1,False,Dragon
214,199,Slowking,Dragon,Psychic,490,95,75,80,100,110,30,2,False,Dragon
218,203,Girafarig,Dragon,Psychic,455,70,80,65,90,65,85,2,False,Dragon
257,238,Smoochum,Dragon,Psychic,305,45,30,15,85,65,65,2,False,Dragon
